In [ ]:
#| hide
from faststripe.core import *

# Tutorial: Get Started with FastStripe

> Learn to accept payments with FastStripe in 15 minutes

This tutorial will guide you through setting up FastStripe and accepting your first payment. By the end, you'll have created a working payment link and understand the basics of the FastStripe API.

## What you'll learn

- How to install and configure FastStripe
- How to create your first payment link
- How to set up a subscription
- How to handle customer data

## Prerequisites

Before starting this tutorial, you'll need:

- Python 3.9 or higher installed
- A Stripe account (sign up at [stripe.com](https://stripe.com))
- Your Stripe test API keys from the [Stripe Dashboard](https://dashboard.stripe.com/test/apikeys)

## Why FastStripe?

FastStripe offers several advantages over the official Stripe Python SDK:

- **Self-documenting**: See all available parameters with descriptions in your IDE
- **Simplified workflows**: High-level methods for common payment patterns
- **Lightweight**: Minimal dependencies (just `fastcore`)
- **Consistent API**: HTTP verb-based methods (`post`, `get`) with full parameter visibility

## Step 1: Installation

First, install FastStripe using pip:

```bash
pip install faststripe
```

Or install the latest development version:

```bash
pip install git+https://github.com/AnswerDotAI/faststripe.git
```

## Versioning

FastStripe versions follow Stripe's API versioning scheme (e.g., `2025.05.28.x`). Each FastStripe release is pinned to a specific Stripe API version, ensuring:

- **Stability**: Your code won't break when Stripe updates their API
- **Predictability**: Same behavior across all environments  
- **Compatibility**: Choose the Stripe API version that works for your application

When you install FastStripe, you get a specific snapshot of the Stripe API that's been tested and validated. The minor version represents non-breaking changes we add such as better higher-level APIs.

## Step 2: Set up your API key

For this tutorial, you'll use your Stripe test API key. Create a `.env` file in your project directory:

```bash
echo "STRIPE_SECRET_KEY=sk_test_your_test_key_here" > .env
```

Then load it in your Python environment:

## Step 3: Initialize FastStripe

Now let's import FastStripe and initialize it with your API key:

In [ ]:
from faststripe.core import StripeApi

import os

# Initialize with your API key from environment
sapi = StripeApi('your-api-key')

In [ ]:
sapi.customers.post(

In [ ]:
# Create a customer
customer = sapi.customers.post(email='user@example.com', name='John Doe')
print(customer.id, customer.email)

cus_ScUPG9yb5cPV6G user@example.com


### Self-Documenting API

One of FastStripe's key advantages is that all methods include parameter documentation directly in your IDE. You can see what parameters are available without checking external docs:

```python
# Explore available methods and their parameters
sapi.customers.post?
```
```
Signature:     
sapi.customers.post(
    address: object = None,
    balance: int = None,
    cash_balance: dict = None,
    description: str = None,
    email: str = None,
    ...
```

It also supports tab completion when filling in parameters!

### High-Level Convenience Methods

FastStripe includes simplified methods for common payment workflows:

In [ ]:
# Create a one-time payment checkout session
checkout = sapi.one_time_payment(
    product_name='My Product',
    amount_cents=2000,  # $20.00
    success_url='https://localhost:8000/success',
    cancel_url='https://localhost:8000/cancel'
)
print(f"Payment URL: {checkout.url[:64]}...")

Payment URL: https://billing.answer.ai/c/pay/cs_test_a107uQXcqI6W9iD09wOmVinc...


In [ ]:
# Create a subscription checkout session
subscription = sapi.subscription(
    product_name='Monthly Plan',
    amount_cents=999,  # $9.99/month
    success_url='https://localhost:8000/success',
    cancel_url='https://localhost:8000/cancel',
    customer_email=customer.email
)
print(f"Subscription URL: {subscription.url[:64]}...")

Subscription URL: https://billing.answer.ai/c/pay/cs_test_a1O4fjw1mgs11zkLGgHZTp6T...


### Complete API Coverage

FastStripe provides access to the entire Stripe API through organized resource groups:

In [ ]:
# Access any Stripe resource with consistent patterns
product = sapi.products.post(name='New Product')
print(f"Created product: {product.name} with ID: {product.id}")

Created product: New Product with ID: prod_ScUPzNzla8KDC6


In [ ]:
# Fetch existing resources
customers = sapi.customers.get(limit=3)
print(f"Found {len(customers.data)} customers")

Found 3 customers


In [ ]:
# All responses are AttrDict objects for easy dot notation access
payment_intent = sapi.payment.intents_post(amount=1000, currency='usd')
print(f"Payment intent status: {payment_intent.status}, amount: ${payment_intent.amount/100}")

Payment intent status: requires_payment_method, amount: $10.0


### Pagination Support

FastStripe includes built-in utilities for handling paginated API responses, making it easy to work with large requests.

In [ ]:
from faststripe.page import paged, pages

for p in paged(sapi.customers.get, limit=5): break
print(f"Got {len(p.data)} customers")
print(f"Has more pages: {p.has_more}")

Got 5 customers
Has more pages: True


In [ ]:
sapi.products

- [products.get](https://docs.stripe.com/api/products/list)(active: 'str', created: 'str', ending_before: 'str', expand: 'str', ids: 'str', limit: 'str', shippable: 'str', starting_after: 'str', url: 'str'): *List all products*
- [products.post](https://docs.stripe.com/api/products/create)(active: bool = None, default_price_data: dict = None, description: str = None, expand: list = None, id: str = None, images: list = None, marketing_features: list = None, metadata: dict = None, name: str = None, package_dimensions: dict = None, shippable: bool = None, statement_descriptor: str = None, tax_code: str = None, unit_label: str = None, url: str = None): *Create a product*
- [products.search_get](https://docs.stripe.com/api/searchs/retrieve)(expand: 'str', limit: 'str', page: 'str', query: 'str'): *Search products*
- [products.id_delete](https://docs.stripe.com/api/products/delete)(id): *Delete a product*
- [products.id_get](https://docs.stripe.com/api/products/delete)(id, expand: 'str'): *Retrieve a product*
- [products.id_post](https://docs.stripe.com/api/products/update)(id, active: bool = None, default_price: str = None, description: object = None, expand: list = None, images: object = None, marketing_features: object = None, metadata: object = None, name: str = None, package_dimensions: object = None, shippable: bool = None, statement_descriptor: str = None, tax_code: object = None, unit_label: object = None, url: object = None): *Update a product*
- [products.product_features_get](https://docs.stripe.com/api/features/delete)(product, ending_before: 'str', expand: 'str', limit: 'str', starting_after: 'str'): *List all features attached to a product*
- [products.product_features_post](https://docs.stripe.com/api/features/update)(product, entitlement_feature: str = None, expand: list = None): *Attach a feature to a product*
- [products.product_features_id_delete](https://docs.stripe.com/api/features/delete)(product, id): *Remove a feature from a product*
- [products.product_features_id_get](https://docs.stripe.com/api/features/delete)(product, id, expand: 'str'): *Retrieve a product_feature*

In [ ]:
products = pages(sapi.products.get, limit=10)
len(products), products[0]


(
    650,
    {
        'id': 'prod_ScUPzNzla8KDC6',
        'object': 'product',
        'active': True,
        'attributes': [],
        'created': 1751657895,
        'default_price': None,
        'description': None,
        'images': [],
        'livemode': False,
        'marketing_features': [],
        'metadata': {},
        'name': 'New Product',
        'package_dimensions': None,
        'shippable': None,
        'statement_descriptor': None,
        'tax_code': None,
        'type': 'service',
        'unit_label': None,
        'updated': 1751657895,
        'url': None
    }
)

The pagination utilities work with any Stripe resource that supports pagination:

- **`paged()`**: Creates a paged generator for a resource's API
- **`pages()`**: Iterator that automatically fetches all pages and returns all items returned in those pages

This makes it easy to process large datasets without manually handling pagination tokens.